# Risky Projects

Identify projects that are at risk for going overbudget. A project is considered to be overbudget if the cost of all employees assigned to the project is greater than the budget of the project. 

You'll need to prorate the cost of the employees to the duration of the project. For example, if the budget for a project that takes half a year to complete is $10K, then the total half-year salary of all employees assigned to the project should not exceed $10K. Salary is defined on a yearly basis, so be careful how to calculate salaries for the projects that last less or more than one year.

Output a list of projects that are overbudget with their project name, project budget, and prorated total employee expense (rounded to the next dollar amount).

# Approaches

- To calculate the expense of a project, you'll need to determine the project length in days and prorate employees salaries for respective project duration

- The budget is already given at the project level

- You're only concerned with projects that have employees and budget information so you can join all three tables by an INNER JOIN to get all the required details

- Calculate expense (sum of salaries of all employees aligned to the respective project) at project level using group by 

- Merge expense table with the combined table created in above step where project budget and duration can be found

- Select all the projects where expense is higher budget

In [1]:
# Import your libraries
from getdata_fromhtml import getdata_fromhtml as g
import pandas as pd
import datetime as dt
import numpy as np

# Start writing code
path1 = './data/linkedin_projects.html'
path2 = './data/linkedin_emp_projects.html'
path3 = './data/linkedin_employees.html'

linkedin_projects = g(path1)
linkedin_emp_projects = g(path2)
linkedin_employees = g(path3)

merged1 = pd.merge(linkedin_projects, linkedin_emp_projects, left_on = 'id', right_on = 'project_id', how= 'inner')
merged = pd.merge(merged1, linkedin_employees, left_on = 'emp_id', right_on = 'id', how = 'inner')

merged[['salary', 'budget']] = merged[['salary', 'budget']].astype(float)
merged[['start_date', 'end_date']] = merged[['start_date', 'end_date']].apply(pd.to_datetime)

merged['salary_during'] = merged['salary'] * (merged['end_date'] - merged['start_date']).dt.days/365 
merged
merged = merged.groupby(['title', 'budget'])['salary_during'].sum().reset_index()
merged['salary_total'] = merged['salary_during'].apply(np.ceil)
 
merged['diff'] = merged['budget'] - merged['salary_total']
risky_projects = merged[merged['diff'] < 0]
result = risky_projects[['title', 'budget', 'salary_total']]
result.head()

,title,budget,salary_total
0,Project1,29498.0,36293.0
2,Project11,11705.0,31606.0
3,Project12,10468.0,62843.0
5,Project14,30014.0,36774.0
7,Project16,19922.0,21875.0


# Highest Energy Consumption

Find the date with the highest total energy consumption from the Meta/Facebook data centers. Output the date along with the total energy consumption across all data centers.

# Approaches

- Combine together all three tables using concat()

- Sum the consumption values by date 

- Find the max consumption value and then filter on the max value to find the dates of max consumption

In [2]:
import pandas as pd
from getdata_fromhtml import getdata_fromhtml as g

path1 = './data/fb_eu_energy.html'
path2 = './data/fb_asia_energy.html'
path3 = './data/fb_na_energy.html'

fb_eu_energy = g(path1)
fb_asia_energy = g(path2)
fb_na_energy = g(path3)

df = pd.concat([fb_eu_energy, fb_asia_energy, fb_na_energy])
df['consumption'] = df['consumption'].astype(float)
df = df.groupby('date')['consumption'].sum().to_frame('total').reset_index()

df[df['total'] == df['total'].max()][['date', 'total']]

,date,total
5,2020-01-06,1250.0
6,2020-01-07,1250.0


# Counting Instances in Text

Find the number of times the words 'bull' and 'bear' occur in the contents. We're counting the number of times the words occur so words like 'bullish' should not be included in our count.


Output the word 'bull' and 'bear' along with the corresponding number of occurrences.

# Approache

- Create a new dataframe using pd.DataFrame with columns = ['word','netry'] 


- Where in word column contains the words we want to look for 'bull', 'bear' and netry with the count of each word using str.count('pattern')


- Use sum() to get the total occurences per word

In [3]:
# Import your libraries
import pandas as pd

# Start writing code
df = g('./data/google_file.html') 
data = {'word' : ['bull', 'bear'], 'netry' : [df.contents.str.count(' bull ').sum(), df.contents.str.contains(' bear ').sum()]}

res = pd.DataFrame(data, columns = ['word', 'netry'])
res 

,word,netry
0,bull,3
1,bear,2


# Count the number of user events performed by MacBookPro users

Count the number of user events performed by MacBookPro users.
Output the result along with the event name.


Sort the result based on the event count in the descending order.

# Approache

- Select specific column/s from dataframe using [column_anme] then select rows with values equal to ‘==’ macbook pro

- Use .groupby(column_name) to group the dataframe about the specifed column

- Use size() to get the number of elements in the specified column

- Use sort_values(column_name, order) to sort along a specified column; Set order to False to display the printed values in descending order 

- Use .rename(columns = {'old_name':'new_name'}) to rename specific columns

In [4]:
# Import your libraries
import pandas as pd

# Start writing code
df = g('./data/playbook_events.html')

df = df[df['device'] == 'macbook pro']
res = df.groupby('event_name').size().to_frame('n_userevents').reset_index()

res.head()

,event_name,n_userevents
0,home_page,8
1,like_message,3
2,login,3
3,search_autocomplete,3
4,send_message,1


# Find libraries who haven't provided the email address in 2016 but their notice preference definition is set to email

Find libraries who haven't provided the email address in 2016 but their notice preference definition is set to email.


Output the library code.

# Approaches

- Select specific column/s from dataframe using [column_name] then select rows with values equal to ‘==’ email,False, and 2016


- Use AND '&' to check if conditions are satisfied (True)


- Use [ [ column_name/s] ] to return a specified column of the dataframe


In [5]:
# Import your libraries
import pandas as pd

# Start writing code
df = g('./data/library_usage.html') 
df = df.replace(r'^\s*$', 'NaN', regex = True)
df = df[(df['circulation_active_year'] == 2016) &  (df['notice_preference_definition'].astype(str) == 'email') &  (df['provided_email_address'].astype(str) == False)]

resu = df['home_library_code'].astype(str)
resu
 

Series([], Name: home_library_code, dtype: object)

# Top Ranked Songs

Find songs that have ranked in the top position. Output the track name and the number of times it ranked at the top. Sort your records by the number of times the song was in the top position in descending order.

# Approaches

- Select specific column/s from dataframe using [column_name] then select rows with values equal to ‘==’ 1

- Use .groupby(column_name) to group the dataframe about the specifed column then size() to get the number of elements or count per group

- Convert the resulting object to a dataframe using to_frame()

- Use sort_values(column_name, order) to sort along a specified column; Set order to False to display the printed values in descending order


In [6]:
# Import your libraries
import pandas as pd

# Start writing code
df = g('./data/spotify_worldwide_daily_song_ranking.html')

df = df[df['position'].astype(int) == 1]
resu = df.groupby('trackname').size().to_frame('n_topposition').reset_index().sort_values('n_topposition', ascending = False)
resu

,trackname,n_topposition
1,HUMBLE.,7
0,Bad and Boujee (feat. Lil Uzi Vert),1
2,Look What You Made Me Do,1


# Toptal

- Task 1

You would like to find the sentance containing the largest number of words in some given text. The text is specified a strong 
S consisting of N characters: letters, spaces, dots, question marks, and exclamation marks. 

The text can be divided into sentances by spliting it at dots, question marks, and exclamation marks. A sentance can be divided into words by spliting 
it at spaces. A setance without words is valid, but a valid word must contain at least one letter. 


In [7]:
import re
test='hello! How are you? I am fine, thank you. And you?'

l = re.split('[?] |[!] |[.] |\*|\n',test)
x = [len(x.split(' ')) for x in l]

n_max = x.index(max(x))
print(l[n_max])
print(l)
print(x)
n_max

I am fine, thank you
['hello', 'How are you', 'I am fine, thank you', 'And you?']
[1, 3, 5, 2]


2

# Acceptance Rate By Date

What is the overall friend acceptance rate by date? Your output should have the rate of acceptances by the date the request was sent. Order by the earliest date to latest.

Assume that each friend request starts by a user sending (i.e., user_id_sender) a friend request to another user (i.e., user_id_receiver) that's logged in the table with action = 'sent'. If the request is accepted, the table logs action = 'accepted'. If the request is not accepted, no record of action = 'accepted' is logged.

# Approaches

- Assume that each friend request starts by a user sending a friend request that's logged in the table with action = 'sent'

- When a friend request is accepted, a notification is sent to the user that sent the original friend request and action = 'accepted'. This also means that the original user_id_sender is now logged as user_id_accepted and vice versa for the original receiver. You'll need to filter your tables using this logic.

- Create two subsets, one with 'accepted' and other with 'sent' request

- Merge both subsets with sender and receiver key

- Check if sent request is accepted or not across the dates

- Count overall sent request by date and respective acceptances

- Calculate acceptance rate

In [8]:
import pandas as pd

# Start writing code
df = g('./data/fb_friend_request.html')

df_sent = df[df.action == 'sent']
df_accepted = df[df.action == 'accepted']

df = pd.merge(df_sent, df_accepted, how = 'left', left_on = ['user_id_sender', 'user_id_receiver'], right_on = ['user_id_sender', 'user_id_receiver'])
accepted_count = df.groupby('date_x').count().reset_index()

accepted_count['Acceptance_rate'] = accepted_count['action_y']/accepted_count['action_x']
result = accepted_count[['date_x', 'Acceptance_rate']]
result.head()

,date_x,Acceptance_rate
0,2020-01-04,0.750000
1,2020-01-06,0.666667


# Popularity Percentage

Find the popularity percentage for each user on Meta/Facebook. The popularity percentage is defined as the total number of friends the user has divided by the total number of users on the platform, then converted into a percentage by multiplying by 100.

Output each user along with their popularity percentage. Order records in ascending order by user id.

The 'user1' and 'user2' column are pairs of friends.

# Approaches 

- You’ll need to create two subqueries or CTEs to calculate total unique friends-pair and total number of friend per user

- To calculate total unique friends-pair, UNION the user1 and user2 columns. This will de-duplicate any users that are repeated in the column. You can ensure uniqueness by adding a DISTINCT. To calculate the total count of unique friends-pair, perform a count(*) on the subquery.

- To calculate the total number of friends per user, create one column that contains all users and a second column that contains all their friends. You can accomplish this by user1, user2 UNION user2, user1.

- JOIN the two subqueries together so that you can calculate the percentage of friends over total users in the platform. You’ll need to JOIN using a 1=1 relation as the key. The resulting table will be a list of users, their friends, and total number of users on the platform.

- Lastly, implement the percentage popularity formula by counting the number of friends per user and dividing by the total number of users on the platform.

In [9]:
import pandas as pd
import numpy as np

# Start writing code
df = g('./data/facebook_friends.html')

arr = len(np.unique(np.concatenate([df.user1.values, df.user2.values])))
new_df = df.rename(columns = {'user1': 'user2', 'user2': 'user1'})
final = pd.concat([df, new_df], sort = False).drop_duplicates()
res = final.groupby('user1').size().to_frame('count').reset_index()
res['popularity_percentage'] = (res['count']/arr)*100
res.head()

,user1,count,popularity_percentage
0,1,5,55.555556
1,2,3,33.333333
2,3,3,33.333333
3,4,1,11.111111
4,5,1,11.111111


# Users By Average Session Time
Calculate each user's average session time. A session is defined as the time difference between a page_load and page_exit. For simplicity, assume a user has only 1 session per day and if there are multiple of the same events on that day, consider only the latest page_load and earliest page_exit. Output the user_id and their average session time.

# Approaches

-  Select from the table only events that are crucial for the analysis: page load and exit

- Consider different cases that appear in the data set

- For users loading the page multiple times, assume that there is a unique page visit within the hour.

In [10]:
import pandas as pd

# Start writing code
df = g('./data/facebook_web_logs.html')
# a page load  | 
df['timestamp'], df['user_id'] = df['timestamp'].apply(pd.to_datetime), df['user_id'].astype(int)

df_load = df[(df['action'] == 'page_load')]
df_load['day'] = df['timestamp'].dt.day
df_load = df_load.groupby(['user_id', 'day'])['timestamp'].max().reset_index()

# an exit 
df_exit = df[(df['action'] == 'page_exit')]
df_exit['day'] = df_exit['timestamp'].dt.day
#df_exit['day'] = df_exit['day'].astype(int)

df_exit = df_exit.groupby(['user_id', 'day'])['timestamp'].min().reset_index()

df_new = pd.merge(df_load, df_exit, on = ['user_id','day'], suffixes=['_page_load', '_page_exit'])

df_new['session_time'] = (df_new.timestamp_page_exit - df_new.timestamp_page_load).dt.seconds
df_new = df_new.groupby('user_id')['session_time'].mean().reset_index()
df_new

/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/lib/python3/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,session_time
0,0,1883.5
1,1,35.0


# Premium vs Freemium
  - Hard

Find the total number of downloads for paying and non-paying users by date. Include only records where non-paying customers have more downloads than paying customers. The output should be sorted by earliest date first and contain 3 columns date, non-paying downloads, paying downloads.

# Approaches

- ms_user_dimension will help link the number of downloads with whether an account is paying or non-paying

- Join ms_user_dimension, ms_acc_dimension, ms_download facts using LEFT JOIN

- Create columns for total downloads for non paying customers and paying customers using PIVOT

- The output should contain 3 columns date, non paying downloads and paying downloads and output shoud be sorted by date in ascending order.

In [11]:
import pandas as pd

# Start writing code
ms_user_dimension = g('./data/ms_user_dimension.html')
ms_acc_dimension  = g('./data/ms_acc_dimension.html')
ms_download_facts = g('./data/ms_download_facts.html')

merge1 = pd.merge(ms_user_dimension, ms_acc_dimension, left_on = 'acc_id', right_on = 'acc_id', how= 'inner')

merged = pd.merge(merge1, ms_download_facts, on = 'user_id', how = 'inner')

merged = merged.groupby(['user_id', 'date', 'paying_customer'])['downloads'].sum().reset_index() 
x=merged.pivot_table(index=['date'],columns=['paying_customer'],values=['downloads'],aggfunc='sum')
df = pd.DataFrame(x.to_records())
df.columns = df.columns.str.replace('[()]', '').str.replace('[,]', '').str.replace('downloads', '').str.replace("[' ']", "")
df[df.no > df.yes][['date', 'no', 'yes']].sort_values('date').head()

#ms_user_dimension

,date,no,yes
0,2020-08-15,56,36622
2,2020-08-17,816244785,81
3,2020-08-18,523,52
4,2020-08-19,9103,67
5,2020-08-20,5602,08776


-------------------------------------
# Popularity of Hack
- Easy

Meta/Facebook has developed a new programing language called Hack.To measure the popularity of Hack they ran a survey with their employees. The survey included data on previous programing familiarity as well as the number of years of experience, age, gender and most importantly satisfaction with Hack. Due to an error location data was not collected, but your supervisor demands a report showing average popularity of Hack by office location. Luckily the user IDs of employees completing the surveys were stored.

Based on the above, find the average popularity of the Hack per office location. Output the location along with the average popularity.

# Approaches
- Perform an inner merge on dataframe using pd.merge(df1,df2, left_on = common_key_left_df, right_on = common_key_right_table, how = 'inner')

- Use .groupby(column_name) to group the dataframe about the specifed column and use mean() to get the average value per group

In [12]:
facebook_employees = g('./data/facebook_employees.html')
facebook_hack_survey = g('./data/facebook_hack_survey.html')

df = pd.merge(facebook_employees, facebook_hack_survey, left_on = 'id', right_on = 'employee_id', how = 'inner')
df[['id', 'employee_id']] = df[['id', 'employee_id']].astype(int)
df['popularity'] = df['popularity'].astype(float)


merged_df1 = df.groupby('location')['popularity'].mean().reset_index()
merged_df1.head()

,location,popularity
0,India,7.500000
1,Switzerland,1.000000
2,UK,4.333333
3,USA,4.600000


----------------------------------
# Most Active Users On Messenger
- Medium

Meta/Facebook Messenger stores the number of messages between users in a table named 'fb_messages'. In this table 'user1' is the sender, 'user2' is the receiver, and 'msg_count' is the number of messages exchanged between them.

Find the top 10 most active users on Meta/Facebook Messenger by counting their total number of messages sent and received. Your solution should output usernames and the count of the total messages they sent or received

# Approach
- Concatenate two slices of the dataframe

- Aggregated msg_counts by all the users using group by function

- Sort users by their total number of messages in descending order


In [13]:
import pandas as pd
import datetime as dt


df = g('./data/fb_messages.html')
df['date'], df['msg_count'] = df['date'].apply(pd.to_datetime), df['msg_count'].astype(int)


df_user1 = df[['date', 'user1', 'msg_count']]
df_user2 = df[['date', 'user2', 'msg_count']]
df = pd.concat([df_user1, df_user2.rename(columns = {'user2':'user1'})], ignore_index = True)

df = df.groupby('user1')['msg_count'].sum().to_frame('tota_no_mesa').reset_index()
df['ranking'] = df.tota_no_mesa.rank(method = 'first', ascending = False)

resul = df[df['ranking']<= 10]
resul = resul[['user1', 'tota_no_mesa']]
resul.head()

,user1,tota_no_mesa
6,craig23,43
9,herringcarlos,37
14,jennifer11,28
17,johnmccann,47
23,lfisher,29


----------------
# Spam Posts
-Medium 

Calculate the percentage of spam posts in all viewed posts by day. A post is considered a spam if a string "spam" is inside keywords of the post. Note that the facebook_posts table stores all posts posted by users. The facebook_post_views table is an action table denoting if a user has viewed a post.

# Approach
- Create a separate column in order to mark spam posts

- Use inner merges to find spam posts and the number of total viewed posts in order to calculate the spam viewed ratio per day.

- Group by with two operations at time: sum of spam (assuming spam is 1 and non-spam is 0) and count using agg method with dictionary as argument

In [14]:
import pandas as pd

# Start writing code
facebook_posts = g('./data/facebook_posts.html')
facebook_post_views = g('./data/facebook_post_views.html')
merged = pd.merge(facebook_posts, facebook_post_views, on = 'post_id', how = 'inner')
#merged['date'] = merged['post_date'].dt.to_period('D')
merged_spam = merged[merged['post_keywords'].str.contains('spam')]
merged['post_date'] = merged['post_date'].apply(pd.to_datetime)

df= merged.groupby('post_date')['post_keywords'].count().to_frame('tot_cnt').reset_index()
df_spam = merged_spam.groupby('post_date')['post_keywords'].count().to_frame('spa_cnt').reset_index()

df['percentage'] = 100*df_spam['spa_cnt']/df['tot_cnt']
resu = df[['post_date', 'percentage']]

resu.head()

,post_date,percentage
0,2019-01-01,100.0
1,2019-01-02,50.0


----------------------
# Comments Distribution
- Hard 

Write a query to calculate the distribution of comments by the count of users that joined Meta/Facebook between 2018 and 2020, for the month of January 2020. 

The output should contain a count of comments and the corresponding number of users that made that number of comments in Jan-2020. For example, you'll be counting how many users made 1 comment, 2 comments, 3 comments, 4 comments, etc in Jan-2020. Your left column in the output will be the number of comments while your right column in the output will be the number of users. Sort the output from the least number of comments to highest.

To add some complexity, there might be a bug where an user post is dated before the user join date. You'll want to remove these posts from the result.

# Approach

- Join users and comments table using an inner merge

- Filter the dataset for comments from 01-2020 

- Filter the dataset for users that registered between 2018 and 2020.

- Exclude potential bugs from the dataset where comments that are created before user joined should be excluded

- Count number of comments per user for Jan-2020

- The output should be one column with the count of comments and another column with a count of users that made a certain number of comments in Jan-2020

In [15]:
path1 = './data/fb_users.html'
path2 = './data/fb_comments.html'

fb_users = g(path1)
fb_comments = g(path2)

merged = pd.merge(fb_users, fb_comments, left_on = 'id', right_on = 'user_id', how = 'inner')

#
df = merged
df_jan = df[df['created_at'].between('2020-01-01', '2020-01-31')]
df_jan = df_jan[df_jan['joined_at'].between('2018-01-01', '2020-12-31')]

df_jan = df_jan[df_jan['created_at'] > df_jan['joined_at']]

#df_jan['cnt'] = 1
df_jan = df_jan.groupby('user_id')['body'].count().to_frame('cnt_comments').reset_index()

df_fin = df_jan.groupby('cnt_comments')['user_id'].count().reset_index()
resul = df_fin.sort_values('cnt_comments')

resul.head()

,cnt_comments,user_id
0,1,4
1,2,6
2,3,1
3,4,1
4,6,1


--------------------------------------
# Expensive Projects
- Easy (Microsoft)

Given a list of projects and employees mapped to each project, calculate by the amount of project budget allocated to each employee. The output should include the project title and the project budget per employee rounded to the closest integer. Order your list by projects with the highest budget per employee first.

# Approach
- To calculate the amount of budget allocated to each employee (budget-to-employee ratio), you'll need to determine the project budget and divide by the number of employees on the project

- Remove projects with no employees by linking the two tables and filtering using an INNER JOIN

In [16]:
from getdata_fromhtml import getdata_fromhtml as g

ms_projects = g('./data/ms_projects.html')
ms_emp_projects = g('./data/ms_emp_projects.html')
merged = pd.merge(ms_projects, ms_emp_projects, left_on = 'id', right_on = 'project_id', how = 'inner')

merged[['emp_id', 'project_id']] = merged[['emp_id', 'project_id']].astype(int)
merged['budget'] = merged['budget'].astype(float)

merged['n_emp'] = merged.groupby('title')['emp_id'].transform('count')#.reset_index()
merged['budget_to_emp_ratio'] = merged['budget']/merged['n_emp']
resu = merged[['title', 'budget_to_emp_ratio']]
resu = resu.sort_values('budget_to_emp_ratio', ascending = False).drop_duplicates()
resu = round(resu)
resu.head()

,title,budget_to_emp_ratio
15,Project8,24642.0
97,Project49,24387.0
28,Project15,24058.0
18,Project10,23794.0
37,Project19,22493.0


----------------------
# Cum Sum Energy Consumption
- Hard

Calculate the running total (i.e., cumulative sum) energy consumption of the Meta/Facebook data centers in all 3 continents by the date. Output the date, running total energy consumption, and running total percentage rounded to the nearest whole number.

# Approach
- Think about which method fits best in this exercise: append, join, merge, concat

- Use sum of consumption as an additional column

- Use cumsum() to get the cumulative sum

- Use [ [column_name] ] to return a specified column of the dataframe

- Use to_datetime() to convert timestamp to date format

In [17]:
from getdata_fromhtml import getdata_fromhtml as g
import datetime as dt

fb_eu_energy = g('./data/fb_eu_energy.html')
fb_na_energy = g('./data/fb_na_energy.html')
fb_asia_energy = g('./data/fb_asia_energy.html')

merged = pd.concat([fb_eu_energy, fb_na_energy, fb_asia_energy])
merged['date'], merged['consumption'] = merged['date'].apply(pd.to_datetime), merged['consumption'].astype(float)

merged  = merged.groupby('date')['consumption'].sum().to_frame('total_consumption').reset_index()
merged['cum_sum'] = merged['total_consumption'].cumsum()
 
tota = merged['total_consumption'].sum()

merged['percentage'] =  round(100*merged['cum_sum']/tota)#.round()
merged['date'] = pd.to_datetime(merged['date'], format = '%Y-%m-%d').dt.strftime('%Y-%m-%d')

result = merged[['date', 'cum_sum', 'percentage']]
result.head()

,date,cum_sum,percentage
0,2020-01-01,1050.0,13.0
1,2020-01-02,2175.0,27.0
2,2020-01-03,3275.0,40.0
3,2020-01-04,4450.0,55.0
4,2020-01-05,5650.0,69.0


-------------------------
# Employee and Manager Salaries
- Medium (dropbox)

Find employees who are earning more than their managers. Output the employee's first name along with the corresponding salary. Find employees who are earning more than their managers. Output the employee's first name along with the corresponding salary.

# Approach
- Perform self join to itself using pd.merge(dataframe1, dataframe2, on = common_table_keys); By default would add '_x' and '_y' suffixes to duplicate columns

- Filter out rows where salary_x is greater than salary_y

- Use [ [ column_name/s] ] to return a specified column of the dataframe

In [19]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('./data/employee.html')
df['salary'] = df['salary'].astype(float)

merged = pd.merge(df, df, right_on = 'id', left_on='manager_id')
merged = merged[merged['salary_x'] > merged['salary_y']][['first_name_x', 'salary_x']]
merged.head()

,first_name_x,salary_x
2,Richerd,250000.0


-------------------
# Time Between Two Events
- Hard 

Meta/Facebook's web logs capture every action from users starting from page loading to page scrolling. Find the user with the least amount of time between a page load and their first scroll down. Your output should include the user id, page load time, first scroll down time, and time between the two events in seconds.

# Approach

- Collect load time and scroll time as separate fields

- Calculate the time difference between them

- Select the lowest variance

- This approach is not sensitive to situation when there are multiple operation of the same kind in row

In [20]:
from getdata_fromhtml import getdata_fromhtml as g
import datetime as dt

df = g('./data/facebook_web_logs.html') 
df['timestamp'] = df['timestamp'].apply(pd.to_datetime)
page_load = df[df['action'] == 'page_load'] 
page_scrol = df[df['action'] == 'scroll_down'] 
merged = pd.merge(page_load, page_scrol, on = 'user_id', how = 'inner')

merged['time_diff'] = (merged['timestamp_y'] - merged['timestamp_x']).dt.seconds

resu = merged[merged['time_diff'] == merged['time_diff'].min()]
resu = resu[['user_id', 'timestamp_x', 'timestamp_y', 'time_diff']]
#resu.drop_duplicates()
resu.head()

,user_id,timestamp_x,timestamp_y,time_diff
30,2,2019-04-25 13:41:21,2019-04-25 13:41:30,9


----------------------
# Day 1 Common Reactions
- Medium

Find the most common reaction for day 1 by counting the number of occurrences for each reaction. Output the reaction alongside its number of occurrences.

# Approach 
- Use .groupby(column_name) to group the dataframe about the specifed column

- Use size() to get the number of elements per group then convert the resulting object to a dataframe using to_frame()

- Use sort_values(column_name, order) to sort along a specified column; Set order to False to display the printed values in descending order 

- Filter the dataframe by maximum value of the occurences

In [21]:
from getdata_fromhtml import getdata_fromhtml as g

df = g('./data/facebook_reactions.html')
df['date_day'] = df['date_day'].apply(pd.to_numeric)
df_d1 = df[df['date_day'] == 1]
res = df_d1.groupby('reaction').size().to_frame('n_reaction').reset_index()
res = res[res['n_reaction'] == res['n_reaction'].max()]
res.head()

,reaction,n_reaction
1,laugh,5
2,like,5


----------------------
# Unique Salaries
- Hard (Twitter)

Find the top three unique salaries for each department. Output the department name and the top 3 unique salaries by each department. Order your results alphabetically by department and then by highest salary to lowest.

# Approach
- Remove duplicates on department and salary by using df.drop_duplicates()

- Use groupby() to group per department and nlargest(3) to return the top 3 highest salary per group

- Convert the resulting object to a dataframe using to_frame()

- Perform left join on the resulting highest salary per group dataframe and the distinct department and salary dataframe by using pd.merge(dataframe1, dataframe2, on = ['salary','department'], how = 'left') to determine the other attributes of the highest salary per department

- Use [ [ column_name/s] ] to return a specified column of the dataframe


In [22]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('./data/twitter_employees.html')
df = df.drop_duplicates() 
df['salary'] = df['salary'].apply(pd.to_numeric)
def get_slice(df):
    x_vals = df['salary'].drop_duplicates().nlargest(3)
    return  x_vals

df = df.groupby('department').apply(get_slice).reset_index()
df.head()

,department,level_1,salary
0,Audit,24,110000
1,Audit,25,100000
2,Audit,27,70000
3,Management,3,250000
4,Management,0,200000


---------------------
# User Email Labels
- Medium (Google)

Find the number of emails received by each user under each built-in email label. The email labels are: 'Promotion', 'Social', and 'Shopping'. Output the user along with the number of promotion, social, and shopping mails count.

# Approach
- Perform inner join on google_emails and google_labels df using pd.merge(df1, df2, left_on = common_key_left_df, right_on = common_key_right_table)

- Use .groupby(column_name) to group the dataframe about the specifed column and use size() to count of values per group. Do this for both dataframes

- Pivot the dataframe using pd.pivot_table with to_user as the index, label as column, and the count as values; specify the aggfunc to 'sum'

- Clean multilevel index caused by the pivot by setting the columns as str using astype(str) then apply reset_index()

In [23]:
from getdata_fromhtml import getdata_fromhtml as g
google_gmail_emails = g('./data/google_gmail_emails.html')
google_gmail_labels = g('./data/google_gmail_labels.html')

df = pd.merge(google_gmail_emails, google_gmail_labels, left_on = 'id', right_on = 'email_id', how = 'inner')
df = df.groupby(['to_user', 'label']).size().to_frame('n_cnt').reset_index()
resu = pd.pivot_table(df, index = 'to_user', columns = 'label', values = 'n_cnt').reset_index()
res = resu[['to_user','Promotion', 'Shopping', 'Social']].fillna(0)
res.head()

label,to_user,Promotion,Shopping,Social
0,114bafadff2d882864,0.0,3.0,2.0
1,157e3e9278e32aba3e,1.0,1.0,2.0
2,2813e59cf6c1ff698e,2.0,2.0,4.0
3,32ded68d89443e808,3.0,0.0,1.0
4,406539987dd9b679c0,2.0,6.0,0.0


---------------------------
# Activity Rank
- Medium (google)

Find the email activity rank for each user. Email activity rank is defined by the total number of emails sent. The user with the highest number of emails sent will have a rank of 1, and so on. Output the user, total emails, and their activity rank. Order records by the total emails in descending order. Sort users with the same number of emails in alphabetical order.

In your rankings, return a unique value (i.e., a unique rank) even if multiple users have the same number of emails.

# Approach
- Use groupby() to group along from_users then use size() to get the count of values per group; convert the resulting object to dataframe using to_frame('column_name')

- Create a column that contains the rank based on the count of emails using rank(), specify parameter method='first' to get unique rank

In [24]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('./data/google_gmail_emails.html')
df = df.groupby('from_user').size().to_frame('cnt').sort_values(['cnt', 'from_user'], ascending = [False, True]).reset_index()
df['ranks'] = df.cnt.rank(method = 'first', ascending = False)
resu = df
resu.head()

,from_user,cnt,ranks
0,32ded68d89443e808,19,1.0
1,ef5fe98c6b9f313075,19,2.0
2,5b8754928306a18b68,18,3.0
3,55e60cfcc9dc49c17e,16,4.0
4,91f59516cb9dee1e88,16,5.0


-------------------
# Common Letters
- Hard (Google)

Find the top 3 most common letters across all the words from both the tables. Output the letter along with the number of occurrences and order records in descending order based on the number of occurrences.

# Approach 
- Take away each word per row and store it in a single column. Do this by using str.split(delimiter) then applying stack() to reshape the result, tolist() to convert the object to a list format. Do this for columns words1, words2 and contents
- Concatenate the list of words per column into a single list using the operator '+'
- Use 'delimeter'.join(list) to merge the values as a single string
- Split the string per letter using list()
- Convert the resulting list containing all the letters to a dataframe using pd.DataFrame(data, columns) then drop all rows with empty values
- Groupby letter then use size() to count the number of values or occurences per group

In [25]:
from getdata_fromhtml import getdata_fromhtml as g

google_file_store = g('./data/google_file.html')
df = g('./data/google_word_lists.html')

import pandas as pd

df_words1 = df['words1'].str.split(',', expand =True)
df_words2 = df['words2'].str.split(',', expand =True)
df_content = google_file_store['contents'].str.split(' ', expand = True)

df_words1  = df_words1.stack()
df_words2  = df_words2.stack()
df_content = df_content.stack()

words1 = ''.join(df_words1).lower()
words2 = ''.join(df_words2).lower()
words3 = ''.join(df_content).lower()
 
merged = words1+words2+words3

word_list = list(merged)
data = pd.DataFrame(word_list, columns = ['col_1'])
top3 = data.groupby('col_1').size().to_frame('n_occurence').reset_index().sort_values('n_occurence', ascending = False)

res = top3.iloc[0:3, :]
res.head()

,col_1,n_occurence
3,a,62
7,e,53
20,t,52
